# Ionospheric phase in InSAR data

The interferometric phase of repeat-pass InSAR interferograms ($\Delta\phi$) 
formed from two SAR acquisitions at $t_i$ and $t_j$ contains different components as follows:

$\Delta\phi = \Delta\phi_{displacement} + \Delta\phi_{geometry} + \Delta\phi_{troposphere} + \Delta\phi_{ionosphere}  + \Delta\phi_{noise}$

* $\Delta\phi_{displacement}$: ground displacement

* $\Delta\phi_{geometry}$: geometrical phase (topography + earht curvature)

* $\Delta\phi_{troposphere}$: tropospheric delay

* $\Delta\phi_{ionosphere}$: ionospheric delay 


The interferometric phase derived with repeat-pass interferometry, may be significantly affected by spatial and temporal variation of the number of free electrons in the ionosphere. 

In the ionosphere, highly energetic solar radiations such as
extreme ultraviolet and X-ray radiation partially ionize the atmosphere’s neutral atoms and molecules forming a mixture of free electrons, ions, and neutral gas molecules around earth. The ionosphere extends approximately from altitudes of 60 to 1000 km, with a maximum electron density at around 300 km.

Propagation of microwave signals through the ionosphere
causes distortions in the InSAR data, including:

* defocusing of SAR images 
* Faraday rotation 
* azimuth offsets (extra shift between SAR images in the satellite along-track (azimuth) direction)
* range delay (or phase advance which can significantly bias the ground displacement signal)


Ionospheric delay also introduces an extra phase component to the SAR interferograms, which if not compensated, decreases the accuracy of InSAR measurements of ground displacement. Along-track TEC gradients cause a phase gradient equivalent to an extra Doppler shift, which translates to a time shift in the azimuth direction. In other words, the ionospheric phase gradient introduces extra azimuth offsets between two SAR images that cannot be predicted with geometrical coregistration techniques.

Ionosphere is a dispersive medium with respect to the
microwave frequencies. In such a dispersive medium, the delay of the microwave signal is inversely proportional
to the signal's frequency. 


![title](docs_iono/iono_1.png)

The dispersive property of ionosphere with respect to the microwave signal, provides an opportunity to separate ionospheric phase (which is dispersive) from non-dispersive phase components (ground displacement, troposphere and geometry). For this we need to have the interferograms at least at two different frequencies. Current and most future missions operate at a single center frequency. However, one can split the range bandwidth and form interferograms with slightly different center frequencies. These interferograms can be combined to estimate the ionospheric phase: 

![title](docs_iono/iono_2.png)

The impact of ionospheric phase on ground displacement derived from InSAR data is a function of TEC variations between two SAR acquisition dates and center frequencies at which the data has been acquired. 

TEC variation, is a function of:
* geographic location (e.g., higher TEC at equatorial belt compared to mid-latitudes)
* time of day (lower TEC at dusk-down time)
* season (more TEC at summer hemisphere than winter hemisphere)
* levels of geomagnetic and solar activities (higher TEC at solar maxima (with a cycle of 11 years))

The ionospheric phase is larger at lower frequencies. Given the same variation of ionosphere TEC and a similar imaging geometry, ionospheric phase at L-band frequency (1.27 GHz) is 
* 3.9 times larger than S-band (2.5 GHz)
* 18.1 times larger than C-band (5.405 GHz) 
* 57.7 times larger than X-band (9.65 GHz)

Here is an example of the impact of ionosphere on InSAR data at equatorial belt: 


![title](docs_iono/iono_Maule.png)

stripmapApp allows to estimate ionospheric phase using the split range-spectrum technique. In the following we go back to the ALOS-1 interferogram which we processed in the stripmapApp session and will discuss the processing steps to estimate ionospheric phase.

# Prepare directories

Importing some python modules and setting up some variables: 

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import gdal

home_dir = os.path.join(os.getenv("HOME"), "work")
PROCESS_DIR = os.path.join(home_dir, "Hawaii_ALOS1")
DATA_DIR =  os.path.join(PROCESS_DIR, "data")

print("home directory: ", home_dir)

Check if the PROCESS_DIR and DATA_DIR already exist. If they don't exist, we create them:

cd to the PROCESS_DIR

In [ ]:
os.chdir(PROCESS_DIR)

In [ ]:
pwd

<br>
<div class="alert alert-info">
<b>Note :</b> 
In this notebook we will work on the same example shown on the stripmapApp notebook. Please make sure that you have gone through the stripmapApp notebook first and have sucessfuly run the processing steps up to split_range_spectrum. In the current notebook we will continue the steps from split_range_spectrum.  

</div>

# Setting up input xml files for processing with stripmapApp

Modify the stripmapApp.xml file as follows to introduce the "do split spectrum" and "do dispersive" options.

### stripmapApp.xml



```xml
<?xml version="1.0" encoding="UTF-8"?>
<stripmapApp>
  <component name="insar">
    <property name="sensor name">ALOS</property>
    <component name="master">
        <catalog>master.xml</catalog>
    </component>
    <component name="slave">
        <catalog>slave.xml</catalog>
    </component>

    <!--
    <property name="demFilename">
        <value>demLat_N18_N21_Lon_W156_W154.dem.wgs84</value>
    </property>
    -->
    <property name="unwrapper name">icu</property>
    
    <property name="do split spectrum">True</property>
      
    <property name="do dispersive">True</property>

      
</component>
</stripmapApp>
```

<br>
<div class="alert alert-info">
<b>Note (do rubbersheetin) :</b> 
There is an optional step for rubbersheeting, which may be required when there is azimuth offsets introduced by ionospheric scintillation. If rubbersheeting is turned on, then a dense azimuth offset map is estimated based on amplitude cross-correlation and is used for more precise resampling.  

```xml
    <property name="do rubbersheeting">True</property>
```    
</div>

# stripmapApp processing steps

To see a full list of processing steps using stripmapApp try the help command:

In [ ]:
!stripmapApp.py --help --steps

The default setting of stripmapApp includes the following steps to generate a geocoded interferogram from raw data or SLC images:

'startup', 'preprocess', 
'formslc',
'verifyDEM', 
'topo', 
'geo2rdr', 
'coarse_resample', 
'misregistration', 
'refined_resample', 
'interferogram', 
'filter', 
'unwrap', 
'geocode'

For ionospheric phase estimation, the following additional steps (in red) are required:

'startup', 'preprocess', 
'formslc',
'verifyDEM', 
'topo', 
'geo2rdr', 
'coarse_resample', 
'misregistration', 
'refined_resample', 
<span style="color:red">split_range_spectrum</span>,
<span style="color:red">sub_band_resample</span>,
'interferogram', 
<span style="color:red">sub_band_interferogram</span>,
'filter', 
<span style="color:red">filter_low_band</span>,
<span style="color:red">filter_high_band</span>,
'unwrap',
<span style="color:red">unwrap_low_band</span>,
<span style="color:red">unwrap_high_band</span>,
<span style="color:red">ionosphere</span>,
,'geocode'

Since the processing steps up to refined_resample, are common for the two workflows with or without ionospheric phase correction, then we need to only run the steps after refined_resample.


### split_range_spectrum

At this step the range spectrum of the master and slve SLC images are split to two sub-bands. The sub-bands are chosen to be 1/3 of the spectrum common between master and slave images. 

In [ ]:
!stripmapApp.py stripmapApp.xml --start=split_range_spectrum --end=split_range_spectrum

A folder named "SplitSpectrum" with two subfolders, "lowband" and "highband" which contain the sub-band master and slave SLCs are added to the working directory.

In [ ]:
ls SplitSpectrum

In [ ]:
ls SplitSpectrum/lowBand/

In [ ]:
ls SplitSpectrum/highBand/

### sub_band_resample

At this the sub-band slave SLCs are coregistered to the sub-band master SLCs. The range azimuth offsets required for coregistration are the same as the full band SLC. 

In [ ]:
!stripmapApp.py stripmapApp.xml --start=sub_band_resample --end=sub_band_resample

After resampling the sub-band SLCs, the "lowband" and "highband" folders are added to the "coregisteredSlc" folder. 

### interferogram, sub-band interferogram

using the full-band SLCs, the low-band and the high-band SLCs, we get three interferograms at full band, low-band and the high-band of the range spectrum. 

In [ ]:
!stripmapApp.py stripmapApp.xml --start=interferogram --end=sub_band_interferogram

This will add the lowBand and highBand folders to the interferogram folder.

In [ ]:
ls interferogram/lowBand/

In [ ]:
ls interferogram/highBand/

### filter_low_band, filter_high_band

To filter the full-band, low-band and high-band interferograms: 

In [ ]:
!stripmapApp.py stripmapApp.xml --start=filter  --end=filter_high_band

Note that the three interferograms (full- low- and high-bands) visually look similar. 

In [ ]:
import gdal
import matplotlib.pyplot as plt

ds = gdal.Open("interferogram/filt_topophase.flat", gdal.GA_ReadOnly)
igram_full_band = ds.GetRasterBand(1).ReadAsArray()
ds = None

ds = gdal.Open("interferogram/lowBand/filt_topophase.flat", gdal.GA_ReadOnly)
igram_low_band = ds.GetRasterBand(1).ReadAsArray()
ds = None

ds = gdal.Open("interferogram/highBand/filt_topophase.flat", gdal.GA_ReadOnly)
igram_high_band = ds.GetRasterBand(1).ReadAsArray()
ds = None

fig = plt.figure(figsize=(18, 16))

ax = fig.add_subplot(1,3,1)
ax.imshow(np.angle(igram_full_band), cmap='hsv')
ax.set_title("full-band")
ax.set_axis_off()

ax = fig.add_subplot(1,3,2)
ax.imshow(np.angle(igram_low_band), cmap='hsv')
ax.set_title("low-band")
ax.set_axis_off()

ax = fig.add_subplot(1,3,3)
ax.imshow(np.angle(igram_high_band), cmap='hsv')
ax.set_title("high-band")
#ax.set_axis_off()


However, due to small difference between the center frequency of the sub-bands full-bands, there is small difference among the three interferograms. See the difference between the full-band and low-band interferograms:

In [ ]:
difference_full_low = igram_full_band*np.conjugate(igram_low_band)
fig = plt.figure(figsize=(18, 16))

ax = fig.add_subplot(1,3,1)
cax = ax.imshow(np.angle(difference_full_low), cmap='jet', vmin = -2.5, vmax =-2)
ax.set_title("difference of full-band and low band interferograms")
ax.set_axis_off()
cbar = fig.colorbar(cax, ticks=[-2.5,-2], orientation='horizontal')

igram_low_band = None
igram_full_band = None
igram_high_band = None
difference_full_low = None


### unwrap_low_band, unwrap_high_band

unwrap the full-band, low-band and high-band interferograms

In [ ]:
!stripmapApp.py stripmapApp.xml --start=unwrap  --end=unwrap_high_band

### ionosphere

This step, uses the low-band and high-bamd unwrapped interferograms to estimate the dispersive and non-dispersive phase components. The disperive phase is related to the ionosphere's TEC variation.

In [ ]:
!stripmapApp.py stripmapApp.xml --start=ionosphere --end=ionosphere

The following plot shows the estimated dispersive and non-dispersive phase components. 

In [ ]:
import gdal
import matplotlib.pyplot as plt
def rewrap(data):
    return data-np.round(data/2./np.pi)*2*np.pi
    
    
ds = gdal.Open("ionosphere/dispersive.bil.filt", gdal.GA_ReadOnly)
iono = ds.GetRasterBand(1).ReadAsArray()
ds = None

ds = gdal.Open("ionosphere/nondispersive.bil.filt", gdal.GA_ReadOnly)
non_dispersive = ds.GetRasterBand(1).ReadAsArray()
ds = None

ds = gdal.Open("ionosphere/mask.bil", gdal.GA_ReadOnly)
mask = ds.GetRasterBand(1).ReadAsArray()
ds = None

fig = plt.figure(figsize=(18, 16))

ax = fig.add_subplot(1,3,1)
ax.imshow(rewrap(iono)*mask, cmap='hsv')
ax.set_title("dispersive (ionospheric phase)")
ax.set_axis_off()

ax = fig.add_subplot(1,3,2)
ax.imshow(rewrap(non_dispersive)*mask, cmap='hsv')
ax.set_title("non-dispersive")
ax.set_axis_off()

iono = None
non_dispersive = None


<br>
<div class="alert alert-info">
<b>Note :</b> 
stripmapApp does not correct the interferogram for ionospheric phase. However, the corrected interferogram can be easily obtained by removing the estimated dispersive phase from the full-band interferogram.

</div>


In [ ]:
ds = gdal.Open("interferogram/filt_topophase.unw", gdal.GA_ReadOnly)
igram = ds.GetRasterBand(2).ReadAsArray()
ds = None

ds = gdal.Open("ionosphere/dispersive.bil.filt", gdal.GA_ReadOnly)
iono = ds.GetRasterBand(1).ReadAsArray()
ds = None

igram_iono_corrected = igram - iono

fig = plt.figure(figsize=(18, 16))

ax = fig.add_subplot(1,3,1)
ax.imshow(rewrap(igram), cmap='hsv')
ax.set_title("before ionospheric phase correction")
ax.set_axis_off()

ax = fig.add_subplot(1,3,2)
ax.imshow(rewrap(igram_iono_corrected)*mask, cmap='hsv')
ax.set_title("after ionospheric phase correction")
ax.set_axis_off()

iono = None
igram = None
igram_iono_corrected = None

### geocoding

By runing the geocode step, the interferogram, the dispersive phase and the mask in the ionosphere folder is geocoded. 

In [ ]:
!stripmapApp.py stripmapApp.xml --start=geocode  --end=geocode

The follwoing plot comapres the geocoded interferogram before and after ionospheric phase estimation. 

In [ ]:
# reading the multi-looked wrapped interferogram
ds = gdal.Open("interferogram/filt_topophase.unw.geo", gdal.GA_ReadOnly)
igram = ds.GetRasterBand(2).ReadAsArray()
ds = None

ds = gdal.Open("ionosphere/dispersive.bil.unwCor.filt.geo", gdal.GA_ReadOnly)
iono = ds.GetRasterBand(1).ReadAsArray()
ds = None

ds = gdal.Open("ionosphere/mask.bil.geo", gdal.GA_ReadOnly)
mask = ds.GetRasterBand(1).ReadAsArray()
ds = None

igram_iono_corrected = (igram - iono)*mask

fig = plt.figure(figsize=(14,12))

ax = fig.add_subplot(1,2,1)

cax = ax.imshow(rewrap(igram), cmap = 'hsv')
ax.set_title("geocoded unwrapped (before ionospheric phase correction)")
ax.set_axis_off()

ax = fig.add_subplot(1,2,2)
cax = ax.imshow(rewrap(igram_iono_corrected), cmap = 'hsv')
ax.set_title("geocoded unwrapped (after ionospheric phase correction)")
ax.set_axis_off()

igram = None
iono = None
igram_iono_corrected = None
mask = None

# Supplementary information

### Low-pass filtering the estimated ionospheric phase

The estimated ionospheric phase from split range-spectrum method is usually noisy. Small decorrelation noise in sub-band interferograms amplifies significantly in the estimated ionospheric phase. Therefore low-pass filtering is usually necessary to reduce noise from the ionospheric noise. The default filter included in stripmapApp is an iterative low-pass filter with a gaussian kernel whose parameters can be controlled from the input xml file. 

```xml
<?xml version="1.0" encoding="UTF-8"?>
<stripmapApp>
  <component name="insar">
    <property name="sensor name">ALOS</property>
    <component name="master">
        <catalog>master.xml</catalog>
    </component>
    <component name="slave">
        <catalog>slave.xml</catalog>
    </component>

    <property name="demFilename">
        <value>demLat_N18_N21_Lon_W156_W154.dem.wgs84</value>
    </property>

    <property name="unwrapper name">icu</property>
    
    
    <property name="do dispersive">True</property>

    <property name="dispersive filter kernel x-size">800</property>
    <property name="dispersive filter kernel y-size">800</property>
    <property name="dispersive filter kernel sigma_x">100</property>
    <property name="dispersive filter kernel sigma_y">100</property>
    <property name="dispersive filter kernel rotation">0</property>
    <property name="dispersive filter number of iterations">5</property>
    <property name="dispersive filter mask type">coherence</property>
    <property name="dispersive filter coherence threshold">0.6</property>
      
</component>
</stripmapApp>
```

![title](docs_iono/Kernel.png)

### azimuth offsets induced by high ionospheric phase gradient

The Doppler frequency induced by high ionospheric phase gradient introduces an extra time shift (azimuth offset) which if not accounted for causes decorrelation. This effect only is observed when ionospheric phase has a spatially high frequency component usually caused by scintillation in the equatorial belt. stripmapApp allows to account for this extra azimuth shift using the optional step "rubber_sheet", which if turned on, extra azimuth shifts are computed using amplitude cross-correlation. Here is an example showing the impact of such ionospher's induced azimuth offsets on an L-band ALOS-1 interferogram in equatorial belt.

![title](docs_iono/AzOffsets.png)